In [1]:
import pandas as pd

In [33]:
ind=pd.read_csv('data/ind.csv',index_col=0,engine='c',usecols=['_'])

In [43]:
files=['SRR3264'+str(i) +'.meth' for i in [413,414,415,416,722]]
n=8
dfs=[]
for f in files:
    print(f,end=',')
    df=pd.read_csv('data/meth/fib/'+f,sep='\t',header=None,nrows=5*10**n)
    df=df[df[5]>0]
    df.index=df[0].str[3:5]+'_'+df[1].astype(str)
    df=df[df.index.isin(ind.index)][[4]]
    print(round(len(df)/10**6))
    df.columns=[f]
    dfs.append(df)    

SRR3264413.meth,3469996,SRR3264414.meth,3458878,SRR3264415.meth,3447692,SRR3264416.meth,3467761,SRR3264722.meth,3437878,

In [44]:
df=dfs[0]
for i in range(1,len(files)):
    print(i,end=',')
    dt=dfs[i]
    df=df.merge(dt,left_index=True,right_index=True)
print(df.shape)    

1,2,3,4,(3150367, 5)


In [45]:
df.to_csv('data/meth/fib.csv')